# Optimization Sandbox

Goal is to find the global "most optimal" points for water sources (destinations of supply) among a large set of homes and corrals (origins of demand). For now, we're minimizing the average traveling time from a cluster of origins to a destiantion.

In [ ]:
import numpy as np
from numpy import random as rand
from scipy.interpolate import make_interp_spline
import matplotlib.pyplot as plt

np.set_printoptions(suppress=True)  # Removes scientific notation, etc.

## Abstract Examples
Experimenting with different optimization methods with arbitrary examples.

Here, we globally optimize a continuous function with local extrema with a continuous bounding function (to determine the minimum a minima can be, etc.).

In [ ]:
def gen_ex_func(num_pts=64) -> tuple[np.ndarray, np.ndarray]:
    """
    Generates a random example function to optimize with exaggerated local extrema, given the number of points to interpolate from
    """
    x_vals = np.arange(num_pts)
    y_vals = rand.gumbel(
        size=num_pts
    )  # Skewed distribution to occasionally generate much larger values for extrema

    spline = make_interp_spline(x_vals, y=y_vals)
    x_vals = np.linspace(0, x_vals.max(), 512)
    return x_vals, spline(x_vals)

In [ ]:
x_vals, y_vals = gen_ex_func()

fig, ax = plt.subplots()
ax.plot(x_vals, y_vals)
ax.plot(x_vals, np.sin(x_vals))

In [ ]:
def annealing(
    random_start,
    cost_function,
    random_neighbour,
    acceptance,
    temperature,
    maxsteps=1000,
    debug=True,
):
    """Optimize the black-box function 'cost_function' with the simulated annealing algorithm."""
    state = random_start()
    cost = cost_function(state)
    states, costs = [state], [cost]

    for step in range(maxsteps):
        fraction = step / float(maxsteps)
        T = temperature(fraction)
        new_state = random_neighbour(state, fraction)
        print(new_state)
        new_cost = cost_function(new_state)
        if debug:
            print(
                "Step #{:>2}/{:>2} : T = {:>4.3g}, state = {:>4.3g}, cost = {:>4.3g}, new_state = {:>4.3g}, new_cost = {:>4.3g} ...".format(
                    step, maxsteps, T, state, cost, new_state, new_cost
                )
            )
        if acceptance(cost, new_cost, T) > rand.random():
            state, cost = new_state, new_cost
            states.append(state)
            costs.append(cost)
            # print("  ==> Accept it!")
        # else:
        #    print("  ==> Reject it...")
    return state, cost_function(state), states, costs

In [ ]:
interval = (0, 512)


def f(x: int):
    """Discrete function of interger points from 0 to 512"""
    return y_vals[x]


def clip(x) -> int:
    """Force point to be in the interval."""
    a, b = interval
    return int(max(min(x, b), a))


def random_start() -> int:
    """Random integer point in the interval."""
    a, b = interval
    return int(a + (b - a) * rand.random_sample())


def cost_function(x: int):
    """Cost of x = f(x)."""
    return f(x)


def random_neighbour(x, fraction=1):
    """Move a little bit x, from the left or the right."""
    amplitude = (max(interval) - min(interval)) * fraction / 10
    delta = (-amplitude / 2.0) + amplitude * rand.random_sample()
    return clip(x + delta)


def acceptance_probability(cost, new_cost, temperature):
    if new_cost < cost:
        # print("    - Acceptance probabilty = 1 as new_cost = {} < cost = {}...".format(new_cost, cost))
        return 1
    else:
        p = np.exp(-(new_cost - cost) / temperature)
        # print("    - Acceptance probabilty = {:.3g}...".format(p))
        return p


def temperature(fraction):
    """Example of temperature dicreasing as the process goes on."""
    return max(0.01, min(1, 1 - fraction))


def see_annealing(states, costs):
    plt.figure()
    plt.suptitle("Evolution of states and costs of the simulated annealing")
    plt.subplot(121)
    plt.plot(states, "r")
    plt.title("States")
    plt.subplot(122)
    plt.plot(costs, "b")
    plt.title("Costs")
    plt.show()

In [ ]:
state, c, states, costs = annealing(
    random_start,
    cost_function,
    random_neighbour,
    acceptance_probability,
    temperature,
    maxsteps=1000,
    debug=True,
)

print(state, c)
see_annealing(states, costs)

## Simulated Annealing
**Note:** `scipy.optimize.anneal` deprecated in favor of `scipy.optimize.basinhopping`.
We're testing this on a randomly generated $32 \times 2$ array that represents 32 random coordinates (each coordinate in $[0, 100)$).

In [ ]:
def gen_rand_coords(num_coords=1, lo=0, up=100) -> tuple[np.ndarray, np.ndarray]:
    """
    Generates random coordinates given number of coordinates within [lower bound, upper bound)
    """
    return rand.rand(num_coords, 2) * up + lo

In [ ]:
origins = gen_rand_coords(100)  # Homes/corrals
dests = gen_rand_coords(16)  # Water sources

In [ ]:
fig, ax = plt.subplots()

ax.scatter(origins[:, 0], origins[:, 1])  # Blue
ax.scatter(dests[:, 0], dests[:, 1])  # Orange

In [ ]:
def avg_dist_from_pt(pt: np.ndarray, arr: np.ndarray) -> float:
    """
    Calculates average distance that an array of points are from a point
    """
    sum = 0
    for arr_pt in arr:
        sum += np.linalg.norm(pt - arr_pt)
    return sum / arr.size

Absolute optimization example (control result to compare against)

In [ ]:
avgs = np.array([avg_dist_from_pt(dest, origins) for dest in dests])

fig, ax = plt.subplots()
ax.scatter(dests[:, 0], dests[:, 1], c=avgs, s=64)  # s is a size of marker
plt.gray()